Ce notebook a été entièrement réalisé par Mélnaie Gomis.

# ViT

In [1]:
# Installation de gdown si besoin
try:
    import gdown
except ImportError:
    %pip install -q gdown
    import gdown


In [ ]:
import numpy as np
import glob
import os
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import gdown
import matplotlib.pyplot as plt

# Chargement des données

In [ ]:
"""
    os.chdir("../Données")
    fichiers_npz = glob.glob('*.npz')
    cosmos_files = [f for f in fichiers_npz if "COSMOS" in f]
"""

In [ ]:
# On définit le nom du dossier
dossier_nom = "redshift"
url = "https://drive.google.com/drive/folders/1-tQH6rfB1XoF7ml98yqVn7Z2IQwGMUdK?usp=sharing"

# 2. LOGIQUE DE TÉLÉCHARGEMENT
if os.path.exists(dossier_nom):
    print(f"✅ Le dossier '{dossier_nom}' est déjà présent. Pas besoin de retélécharger !")
else:
    print(f"📥 Dossier introuvable. Téléchargement en cours...")
    gdown.download_folder(url, quiet=True, use_cookies=False)
    print("✅ Téléchargement terminé !")

# 3. CRÉATION DE LA LISTE DES FICHIERS (C'est ici que ça plantait avant)
# On scanne le dossier pour créer la liste 'dossier_local' quoi qu'il arrive
pattern = os.path.join(dossier_nom, "*") # ex: redshift/*
dossier_local = glob.glob(pattern)

# 4. AFFICHAGE JOLI
print(f"\nSuccès ! {len(dossier_local)} fichiers trouvés dans '{dossier_nom}'.")
print("Détail des fichiers :")

for chemin in dossier_local:
    # On extrait le nom du dossier et le nom du fichier
    dossier = os.path.dirname(chemin)   # ex: redshift
    fichier = os.path.basename(chemin)  # ex: COSMOS_...npz
    
    # Ton format demandé
    print(f"[{dossier}] :\t {fichier}")

In [ ]:
fichiers_npz = glob.glob('redshift/*.npz')
cosmos_files = [f for f in fichiers_npz if "COSMOS" in f]

# Listes séparées pour train et test
train_cubes_list = []
train_infos_list = []
train_flag_list = []

test_cubes_list = []
test_infos_list = []
test_flag_list = []

print(f"Chargement de {len(cosmos_files)} fichiers COSMOS.")

for fichier in cosmos_files:
    try:
        data = np.load(fichier, allow_pickle=True)
        
        # 'info' est un array structuré. 'dtype.names' donne les noms des colonnes.
        info_fields = data['info'].dtype.names
        
        # On vérifie si la *colonne* 'ZSPEC' existe dans ce fichier
        if 'ZSPEC' in info_fields:
            # Ce fichier contient des données de TEST (il a la colonne ZSPEC)
            test_cubes_list.append(data['cube'])
            test_infos_list.append(data['info'])
            test_flag_list.append(data['flag'])
        else:
            # Ce fichier contient des données de TRAIN (pas de colonne ZSPEC)
            train_cubes_list.append(data['cube'])
            train_infos_list.append(data['info'])
            train_flag_list.append(data['flag'])
            
            
    except Exception as e:
        print(f":x: Erreur en chargeant {fichier}: {e}")

print("Chargement terminé.")

In [ ]:
# Création des jeux train/test

# Jeu d’entraînement (basé sur les fichiers SANS ZSPEC)
# On vérifie qu'on a bien trouvé des fichiers de train
if train_infos_list:
    X_train = np.concatenate(train_cubes_list, axis=0)
    infos_train = np.concatenate(train_infos_list, axis=0)
    flags_train = np.concatenate(train_flag_list, axis=0)
    y_train = infos_train['ZPHOT'] # Obtenir ZPHOT depuis les infos de train
    print(f"Objets d'entraînement (ZPHOT) : {len(y_train)}")
    print("X_train :", X_train.shape)
    print("y_train :", y_train.shape)
else:
    print("Aucun fichier d'entraînement (sans ZSPEC) trouvé.")

print("\n---\n")

# Jeu de test (basé sur les fichiers AVEC ZSPEC)
# On vérifie qu'on a bien trouvé des fichiers de test
if test_infos_list:
    X_test = np.concatenate(test_cubes_list, axis=0)
    infos_test = np.concatenate(test_infos_list, axis=0)
    flags_test = np.concatenate(test_flag_list, axis=0)
    y_test = infos_test['ZSPEC'] # Obtenir ZSPEC depuis les infos de test

    # Si on ne garde que les ZSPEC valides
    # mask_zspec_valid = ~np.isnan(y_test) ou ???
    # X_test = X_test[mask_zspec_valid]
    # y_test = y_test[mask_zspec_valid]

    print(f"Objets de test (ZSPEC) : {len(y_test)}")
    print("X_test  :", X_test.shape)
    print("y_test  :", y_test.shape)
else:
    print("Aucun fichier de test (avec ZSPEC) trouvé.")

# Encodeur d’images (Image Encoder)

On veut des patchs précis mais pas trop lourd. 

- Patch embedding 
    - On utilisera des patchs de 16x16 feront l'affaire.
    - La projection se fera dans un embedding dimension de 512.
- Transformers 
    - On prendre 7 couches et 8 têtes (car 512 / 64 = 8).



In [ ]:
X_train_tensor = torch.tensor(np.ascontiguousarray(X_train), dtype=torch.float32)
y_train_tensor = torch.tensor(np.ascontiguousarray(y_train), dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(np.ascontiguousarray(X_test), dtype=torch.float32)
y_test_tensor = torch.tensor(np.ascontiguousarray(y_test), dtype=torch.float32).unsqueeze(1)


In [ ]:
batch_size = 32

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Modèle sur-mesure

In [ ]:
# --- Patch Embedding ---
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=64, patch_size=16, in_channels=9, embed_dim=512):
        super().__init__()
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        self.proj = nn.Linear(patch_size * patch_size * in_channels, embed_dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.randn(1, self.n_patches + 1, embed_dim))

    def forward(self, x):
        B, H, W, C = x.shape
        patches = x.unfold(1, self.patch_size, self.patch_size) \
                   .unfold(2, self.patch_size, self.patch_size)
        patches = patches.contiguous().view(B, -1, self.patch_size * self.patch_size*C)
        patches = self.proj(patches)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, patches), dim=1)
        x = x + self.pos_embed
        return x

# --- Transformer Block ---
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim=512, num_heads=8, mlp_dim=2048, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, embed_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        x_ = self.ln1(x)
        x_attn, _ = self.attn(x_, x_, x_)
        x = x + x_attn
        x = x + self.dropout1(x_attn)
        x = x + self.mlp(self.ln2(x))
        return x

# --- ViT Image Encoder ---
class ViTImageEncoder(nn.Module):
    def __init__(self, img_size=64, patch_size=16, in_channels=9, embed_dim=512,
                 depth=7, num_heads=8, mlp_dim=2048):
        super().__init__()
        self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_dim) for _ in range(depth)
        ])
        self.ln = nn.LayerNorm(embed_dim)

    def forward(self, x):
        x = self.patch_embed(x)
        x = x.permute(1, 0, 2)  # seq_len, batch, embed_dim
        for blk in self.transformer_blocks:
            x = blk(x)
        x = x.permute(1, 0, 2)  # batch, seq_len, embed_dim
        x = self.ln(x)
        cls_embedding = x[:, 0, :]
        return cls_embedding


In [ ]:
""" class ParamMLP(nn.Module): 
    def __init__(self, input_dim=1, embed_dim=512):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, embed_dim)
        )
        
    def forward(self, x):
        return self.fc(x)
 """

In [ ]:
class ViTRegressor(nn.Module):
    def __init__(self, img_size=64, patch_size=16, in_channels=9,
                 embed_dim=512, depth=7, num_heads=8, mlp_dim=2048):
        super().__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer

        # ViT encodeur
        self.vit = ViTImageEncoder(img_size, patch_size, in_channels,
                                   embed_dim, depth, num_heads, mlp_dim)
        
        # couche finale pour prédire ZPHOT
        self.regressor = nn.Linear(embed_dim, 1)
        
    def forward(self, x):
        embedding = self.vit(x)  # B, 512
        zphot_pred = self.regressor(embedding)  # B,1
        return zphot_pred


## Régression

In [ ]:
import torch
import numpy as np

class ViTTrainer:
    def __init__(self, model, optimizer, criterion, device):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device

        self.train_losses = []
        self.test_losses = []

    # Entraînement (LOSS SEULEMENT)
    def train_epoch(self, train_loader):
        self.model.train()
        running_loss = 0.0

        for xb, yb in train_loader:
            xb, yb = xb.to(self.device), yb.to(self.device)

            self.optimizer.zero_grad()
            preds = self.model(xb)
            loss = self.criterion(preds, yb)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item() * xb.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        self.train_losses.append(epoch_loss)
        return epoch_loss

    # Évaluation GÉNÉRIQUE (train OU test)
    def evaluate(self, loader):
        self.model.eval()
        running_loss = 0.0

        all_preds = []
        all_targets = []

        with torch.no_grad():
            for xb, yb in loader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                preds = self.model(xb)

                loss = self.criterion(preds, yb)
                running_loss += loss.item() * xb.size(0)

                all_preds.append(preds.cpu())
                all_targets.append(yb.cpu())

        loss = running_loss / len(loader.dataset)

        z_pred = torch.cat(all_preds).squeeze().numpy()
        z_true = torch.cat(all_targets).squeeze().numpy()

        delta_z = z_pred - z_true
        delta_z_norm = delta_z / (1.0 + z_true)

        biais = np.mean(delta_z_norm)
        sigma_nmad = 1.48 * np.median(
            np.abs(delta_z_norm - np.median(delta_z_norm))
        )

        return loss, biais, sigma_nmad

    # Boucle principale
    def fit(self, train_loader, test_loader, num_epochs):
        for epoch in range(num_epochs):
            train_loss = self.train_epoch(train_loader)

            train_loss_eval, train_biais, train_sigma = self.evaluate(train_loader)
            test_loss, test_biais, test_sigma = self.evaluate(test_loader)

            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Train Loss : {train_loss:.4f}")
            print(f"Test  Loss : {test_loss:.4f}")
            print(f"Train Biais : {train_biais:.5f} | σ_NMAD : {train_sigma:.5f}")
            print(f"Test  Biais : {test_biais:.5f} | σ_NMAD : {test_sigma:.5f}\n")


Une époque prends environs 13.25 secondes avec les GPU T4 de Google Colab.

In [ ]:
num_epochs = 200 # augmenter si nécessaire
tps = 13.25
tps_approx = tps * num_epochs


# Conversion des secondes en jours, heures et minutes
jours = tps_approx // (24 * 3600)
tps_approx %= (24 * 3600)
heures = tps_approx // 3600
tps_approx %= 3600
minutes = tps_approx // 60
secondes = tps_approx % 60


# Affichage
print(f"Temps approximatif : {int(jours)} J {int(heures)} H {int(minutes)} min")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ViTRegressor().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
trainer = ViTTrainer(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    device=device
)

trainer.fit(train_loader, test_loader, num_epochs)

In [ ]:
train_losses = trainer.train_losses
test_losses  = trainer.test_losses
train_biases = trainer.train_biases
test_biases  = trainer.test_biases
train_sigma  = trainer.train_sigma
test_sigma   = trainer.test_sigma

epochs = range(1, len(train_losses)+1)

fig, axs = plt.subplots(2,2, figsize=(12,8))

# --------- 1. Loss ----------
axs[0,0].plot(epochs, train_losses, label='Train Loss', marker='o')
axs[0,0].plot(epochs, test_losses, label='Test Loss', marker='o')
axs[0,0].set_xlabel('Epoch')
axs[0,0].set_ylabel('Loss')
axs[0,0].set_title('Loss')
axs[0,0].grid(True)
axs[0,0].legend()

# --------- 2. σ_NMAD ----------
axs[0,1].plot(epochs, train_sigma, label='Train σ_NMAD', marker='o')
axs[0,1].plot(epochs, test_sigma, label='Test σ_NMAD', marker='o')
axs[0,1].axhline(0.014, color='red', linestyle='--', label='Target')
axs[0,1].set_xlabel('Epoch')
axs[0,1].set_ylabel('σ_NMAD')
axs[0,1].set_title('σ_NMAD')
axs[0,1].grid(True)
axs[0,1].legend()

# --------- 3. Bias ----------
axs[1,0].plot(epochs, train_biases, label='Train Bias', marker='o')
axs[1,0].plot(epochs, test_biases, label='Test Bias', marker='o')
axs[1,0].axhline(0, color='red', linestyle='--', label='Target')
axs[1,0].set_xlabel('Epoch')
axs[1,0].set_ylabel('Bias')
axs[1,0].set_title('Bias')
axs[1,0].grid(True)
axs[1,0].legend()

# --------- 4. Encadré résumé ----------
axs[1,1].axis('off')  # pas d'axes

# Texte résumé avec les valeurs finales
textstr = (
    f"Final Train:\n"
    f"Loss  = {train_losses[-1]:.4f}\n"
    f"σ_NMAD = {train_sigma[-1]:.4f}\n"
    f"Bias   = {train_biases[-1]:.4f}\n\n"
    f"Final Test:\n"
    f"Loss  = {test_losses[-1]:.4f}\n"
    f"σ_NMAD = {test_sigma[-1]:.4f}\n"
    f"Bias   = {test_biases[-1]:.4f}"
)
axs[1,1].text(0.5, 0.5, textstr, fontsize=12, ha='center', va='center',
              bbox=dict(facecolor='lightgray', edgecolor='black', boxstyle='round'))

plt.tight_layout()
plt.show()

## Classification

In [ ]:
# --- Classe d'entraînement pour le modèle ViT ---
class ViTTrainerClassification:
    def __init__(self, num_classes=200, lr=1e-4):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = ViTModel(num_classes=num_classes).to(self.device)
        
        # Définir le critère de perte et l'optimiseur
        self.criterion = nn.CrossEntropyLoss()  # Utilisation de la CrossEntropyLoss pour la classification
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

    def train(self, train_loader, num_epochs=10):
        self.model.train()  # Mettre le modèle en mode entraînement
        for epoch in range(num_epochs):
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                self.optimizer.zero_grad()  # Zéroiser les gradients
                
                outputs = self.model(inputs)  # Passer les entrées dans le modèle
                loss = self.criterion(outputs, labels)  # Calculer la perte
                
                loss.backward()  # Calculer les gradients
                self.optimizer.step()  # Mettre à jour les poids
                
                running_loss += loss.item()
            
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")
    
    def evaluate(self, val_loader):
        self.model.eval()  # Mettre le modèle en mode évaluation
        correct = 0
        total = 0
        with torch.no_grad():  # Ne pas calculer les gradients pour l'évaluation
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs, 1)  # Obtenir la classe avec la probabilité la plus élevée
                
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = correct / total * 100
        print(f"Accuracy: {accuracy:.2f}%")


# Modèle pré-fait

Au lieu de faire une régression, on va faire une classification.

In [ ]:
# Classe pour le modèle ViT
class ViTModel(nn.Module):
    def __init__(self, num_classes=200):  # Paramètre pour spécifier le nombre de classes
        super(ViTModel, self).__init__()
        # Exemple d'utilisation d'un modèle ViT pré-entrainé (par ex., de Hugging Face ou PyTorch)
        # Ici, on suppose l'utilisation d'un modèle préexistant, comme ViT-B/16 de Hugging Face
        self.vit = models.vit_b_16(pretrained=True)  # Charger le modèle ViT pré-entrainé
        self.vit.conv_proj = nn.Conv2d(
            9,                 # 🔥 nombre de canaux d'entrée
            768,               # embed dim du ViT-B/16
            kernel_size=16,
            stride=16
        )

        # Remplacer la couche de classification pour correspondre au nombre de classes
        self.vit.head = nn.Linear(self.vit.head.in_features, num_classes)

    def forward(self, x):
        return self.vit(x)

# Classe pour entraîner et gérer le modèle
class ViTTrainer:
    def __init__(self, num_classes=200, lr=1e-4):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = ViTModel(num_classes=num_classes).to(self.device)
        
        # Définir le critère de perte et l'optimiseur
        self.criterion = nn.CrossEntropyLoss()  # Utilisation de la CrossEntropyLoss pour la classification
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        
    def train(self, train_loader, num_epochs=10):
        self.model.train()  # Mettre le modèle en mode entraînement
        for epoch in range(num_epochs):
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                
                self.optimizer.zero_grad()  # Zéroiser les gradients
                
                outputs = self.model(inputs)  # Passer les entrées dans le modèle
                loss = self.criterion(outputs, labels)  # Calculer la perte
                
                loss.backward()  # Calculer les gradients
                self.optimizer.step()  # Mettre à jour les poids
                
                running_loss += loss.item()
            
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")
    
    def evaluate(self, val_loader):
        self.model.eval()  # Mettre le modèle en mode évaluation
        correct = 0
        total = 0
        with torch.no_grad():  # Ne pas calculer les gradients pour l'évaluation
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs, 1)  # Obtenir la classe avec la probabilité la plus élevée
                
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = correct / total * 100
        print(f"Accuracy: {accuracy:.2f}%")

In [ ]:
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}")
    
    # évaluation rapide sur test
    model.eval()
    with torch.no_grad():
        test_loss = 0.0
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            test_loss += criterion(pred, yb).item() * xb.size(0)
        test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print(f"Test Loss: {test_loss:.4f}\n")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(train_losses, label="Train Loss")
plt.plot(test_losses, label="Test Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Courbe de Loss")
plt.legend()
plt.grid(True)
plt.show()
